## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-27-14-38-43 +0000,nyt,"William J. Rutter, Biotech Pioneer of Gene-Bas...",https://www.nytimes.com/2025/07/27/science/wil...
1,2025-07-27-14-35-54 +0000,nyt,11 People Injured in Stabbing at a Michigan Wa...,https://www.nytimes.com/2025/07/26/us/stabbing...
2,2025-07-27-14-35-51 +0000,nyt,Dozens of Wildfires Burn in Greece and Turkey ...,https://www.nytimes.com/2025/07/27/world/europ...
3,2025-07-27-14-35-00 +0000,wsj,At Least 11 People Injured in Michigan Walmart...,https://www.wsj.com/us-news/at-least-11-people...
4,2025-07-27-14-32-36 +0000,nypost,Rubio on 2028 election: ‘I think JD Vance woul...,https://nypost.com/2025/07/27/us-news/rubio-on...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,24
38,gaza,16
117,new,14
214,aid,13
49,thailand,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
66,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...,70
156,2025-07-26-16-09-07 +0000,nypost,"Trump mediates Cambodia-Thailand conflict, dem...",https://nypost.com/2025/07/26/us-news/trump-me...,68
87,2025-07-27-03-14-00 +0000,wsj,Thailand and Cambodia Consider Cease-Fire Afte...,https://www.wsj.com/world/asia/thailand-and-ca...,65
81,2025-07-27-04-07-59 +0000,nyt,Trump Says He Expects Quick Cease-Fire Between...,https://www.nytimes.com/2025/07/26/world/asia/...,60
105,2025-07-27-00-00-00 +0000,wsj,Americans view President Trump’s tax-and-spend...,https://www.wsj.com/politics/policy/trump-wsj-...,59


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
66,70,2025-07-27-07-44-36 +0000,nypost,Thailand and Cambodia agree to cease-fire talk...,https://nypost.com/2025/07/27/world-news/thail...
48,54,2025-07-27-09-02-00 +0000,wsj,How Food Supplies in Gaza Have Dwindled Under ...,https://www.wsj.com/world/middle-east/gaza-pal...
94,33,2025-07-27-01-10-28 +0000,nypost,Illinois landlord who killed 6-year-old Palest...,https://nypost.com/2025/07/26/us-news/illinois...
9,26,2025-07-27-14-13-01 +0000,nypost,Trump should consider pardon for Epstein accom...,https://nypost.com/2025/07/27/us-news/trump-sh...
121,24,2025-07-26-21-55-57 +0000,nypost,Iranian Supreme Leader sleeps and gets ‘high o...,https://nypost.com/2025/07/26/world-news/irani...
151,23,2025-07-26-16-54-10 +0000,nypost,Ukraine fends off major Russian advance on fro...,https://nypost.com/2025/07/26/world-news/ten-c...
114,23,2025-07-26-22-27-46 +0000,nypost,Trump is right to want Powell out at the Feder...,https://nypost.com/2025/07/26/business/trump-i...
98,23,2025-07-27-00-54-09 +0000,nypost,American Airlines flight engulfed in ‘a lot of...,https://nypost.com/2025/07/26/us-news/american...
72,22,2025-07-27-06-27-36 +0000,nypost,Three illegal Salvadorans using dating app to ...,https://nypost.com/2025/07/27/us-news/three-il...
135,22,2025-07-26-19-40-14 +0000,cbc,Israel resumes Gaza aid drops days after criti...,https://www.cbc.ca/news/world/gaza-aid-drops-r...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
